In [1]:
import snap
from datetime import datetime
import matplotlib.pyplot as plt
import util

edgeFile = 'brightkite/loc-brightkite_edges.txt'
checkinFile = 'brightkite/loc-brightkite_totalCheckins.txt'

# Build the Graphs
userGraph, geoGraph, visitGraph, coordinates, checkins = util.buildGraphs(edgeFile, checkinFile)

/Users/chen/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [7]:
userList = []
for node in userGraph.Nodes():
    Id = node.GetId()
    if visitGraph.IsNode(Id):
        userList.append(Id)
dominanceI = []
with open('dominanceI2.txt') as f:
    for line in f: dominanceI.append(int(line.split()[0]))
Eis = {userList[i]:dominanceI[i] for i in xrange(len(userList))}

In [10]:
locList = []
for node in geoGraph.Nodes():
    Id = node.GetId()
    locNI = visitGraph.GetNI(Id)
    deg = locNI.GetDeg()
    if deg != 0:
        locList.append(Id)

entropyI = []
with open('loc_entropyI.txt') as f:
    for line in f: entropyI.append(float(line.split()[0]))
Bis = {locList[i]:entropyI[i] for i in xrange(len(locList))}

In [15]:
import numpy as np
userMeanBi = []
userMedianBi = []
for userId in userList:
    NI = visitGraph.GetNI(userId)
    deg = NI.GetDeg()
    BiList = []
    for k in xrange(deg):
        BiList.append(Bis[NI.GetNbrNId(k)])
    userMeanBi.append(np.average(BiList))
    userMedianBi.append(np.median(BiList))

In [30]:
fig = plt.figure(figsize = (6,6))
p1 = plt.scatter(dominanceI, userMeanBi, s = 0.5, color = '#ff7373', edgecolors = 'face')
p2 = plt.scatter(dominanceI, userMedianBi, s = 0.5, color = '#00ced1', edgecolors = 'face')
plt.legend((p1, p2), ('Mean', 'Median'))
plt.xlabel('E_i for Users')
plt.ylabel("Mean/Median of neighbors' B_i")
plt.savefig('user_loc_test.png')
plt.close()
fig = plt.figure(figsize = (6,6))
p1 = plt.scatter(entropyI, locMeanEi, s = 0.5, color = '#ff7373', edgecolors = 'face')
p2 = plt.scatter(entropyI, locMedianEi, s = 0.5, color = '#00ced1', edgecolors = 'face')
plt.legend((p1, p2), ('Mean', 'Median'))
plt.xlabel('B_i for Places')
plt.ylabel("Mean/Median of neighbors' E_i")
plt.savefig('loc_user_test.png')
plt.close()

In [18]:
from scipy.stats.stats import pearsonr
print pearsonr(dominanceI,userMeanBi)

(-0.15163612681231411, 5.1220846255731259e-262)


In [19]:
print pearsonr(dominanceI,userMedianBi)

(-0.17859827453384519, 0.0)


In [23]:
import numpy as np
locMeanEi = []
locMedianEi = []
for locId in locList:
    NI = visitGraph.GetNI(locId)
    deg = NI.GetDeg()
    EiList = []
    for k in xrange(deg):
        EiList.append(Eis[NI.GetNbrNId(k)])
    locMeanEi.append(np.average(EiList))
    locMedianEi.append(np.median(EiList))

In [28]:
fig = plt.figure(figsize = (6,6))
p1 = plt.scatter(entropyI, locMeanEi, s = 0.5, color = '#ff7373', edgecolors = 'face')
p2 = plt.scatter(entropyI, locMedianEi, s = 0.5, color = '#00ced1', edgecolors = 'face')
plt.legend((p1, p2), ('Mean', 'Median'))
plt.xlabel('B_i for Places')
plt.ylabel("Mean/Median of neighbors' E_i")
plt.savefig('loc_user_test.pdf')
plt.close()

In [26]:
pearsonr(entropyI, locMeanEi)

(0.0092884467140635779, 3.1848525161331999e-16)

In [27]:
pearsonr(entropyI, locMedianEi)

(-0.032285928701726994, 2.5696930630619102e-177)